# Data analysis with pandas

> Objectives:
> * Be able to load and save data using pandas
> * Be able to access columns, rows, and elements in DataFrames and Series objects
> * Be able to perform aggregate computations across different variables
> * Be able to filter and subset data

[pandas](http://pandas.pydata.org/) is a Python library that provides tools for processing and manipulating data.

Typically, you will see pandas imported as "`pd`", which is shorter and therefore easier to type than the full name `pandas`:

In [ ]:
import pandas as pd

## Loading data

First and foremost, pandas gives us a really convenient way to read in data in CSV ("comma separated value") format. In this lesson, we have two CSV files containing information about precipitation in California. The first file we'll take a look at is `precip_data.csv`, which contains monthly aggregate data:

In [ ]:
!head precip_data.csv

To load it, we call `read_csv` function, and pandas automatically figures out how to read the file for us:

In [ ]:
monthly = pd.read_csv("precip_data.csv")
monthly

The type of object that is returned is called a "data frame", and is one of two fundamental data types that pandas uses:

In [ ]:
type(monthly)

A data frame is essentially a table which has labeled rows and columns. By default when reading in a csv file, pandas will create a numerical index for the rows, and will use the first row of the CSV as the column names. However, having a numerical index isn't necessarily what we want. In this case, it might be more useful for us to have the rows correspond to different stations and the dates the data was recorded. To do this, we can use the `set_index` method of the data frame object:

In [ ]:
monthly = monthly.set_index('station')
monthly

## Accessing rows, columns, and elements

To access a column in a DataFrame, we index into the DataFrame as if it were a dictionary. For example, to get just the precipitation for each station and date:

In [ ]:
monthly['precip']

The type of object that is returned is a `Series` object, which is the 1D equivalent of a DataFrame. We can further index into this Series object, for example, to get the precipitation for one particular station:

In [ ]:
monthly['precip']['San Jose']

If we want to access the data the other way around -- i.e., access the row(s) first, and then the `'precip'` column -- we need to index slightly differently, using the `.loc` attribute:

In [ ]:
monthly.loc['San Jose']

This returns another DataFrame, which we can then index as we saw earlier:

In [ ]:
monthly.loc['San Jose']['precip']

To summarize:

In [ ]:
# column indexing --> Series
monthly['precip']

# column, then row indexing --> Series or element
monthly['precip']['San Jose']

# row indexing --> DataFrame or Series
monthly.loc['San Jose']

# row, then column indexing --> Series or element
monthly.loc['San Jose']['precip']

## Saving data

Let's say we want to save out just the San Jose data. We can do this using the `.to_csv()` method of the DataFrame:

In [ ]:
monthly.loc['San Jose'].to_csv('san_jose.csv')

In [ ]:
!head san_jose.csv

## Aggregate computations

One of the really powerful operations that pandas can do involves splitting the data up into particular groups, performing some operation on each group, and then recombining the results.

For example, how would we compute the total precipitation per year for each station? To do this, we want to:

* split the data into groups, where there each group corresponds to one station and year
* sum across the precipitation values for each group
* recombine the resulting sums

In pandas, this is really easy! First, we'll want to "reset" the index of our DataFrame so that everything is in columns:

In [ ]:
monthly = monthly.reset_index()
monthly

Now, we use the `groupby` command to specify which columns should be used to form the groups:

In [ ]:
gb = monthly.groupby(['station', 'year'])
gb

We can index into this "groupby" object just like a DataFrame, and select only the precipitation data:

In [ ]:
gb['precip']

Finally, we can calculate summary statistics on these groups. For example, a sum of the precipitation each year for each station:

In [ ]:
gb['precip'].sum()

The result is a Series object that has as its index labels for the stations and years. The values of the Series objects are the total precipitation for the corresponding station and year.

Similar computations follow the same basic recipe. For example, to compute the *average* precipitation per *month*:

In [ ]:
gb['precip'].mean()

## Filtering Data

We've looked at how to group our data into subsets and how to use get subsets using indexing, but there are also other ways to create subsets of data.
Above we used `.set_index` to move the `'station'` column into the index then grab the data for the `'San Jose'` station. Here's another way to do that using "boolean indexing":

In [ ]:
monthly.loc[monthly.station == 'San Jose']

Another way to filter is with the `.isin` method. This tests whether values are in another set of values.
Say we want all the data for the winter months Dec, Jan, Feb:

In [ ]:
monthly.loc[monthly.month.isin(['Dec', 'Jan', 'Feb'])]

And the `.query` method is useful for more complex queries:

In [ ]:
monthly.query('region == "SAN FRANCISCO BAY" and month in ["Dec", "Jan", "Feb"]')

## Plotting

In [ ]:
%matplotlib inline

Let's start with plot of the mean California rainfall over the years. The first step will be to use groupby ot aggregate the data by year:

In [ ]:
yearly = monthly.groupby('year').precip.mean()
yearly.head()

That gives us a Series, which has a .plot method for plotting the index of the Series vs. the values:

In [ ]:
yearly.plot()

Or we can compare regions and make a bar chart:

In [ ]:
regional = monthly.groupby('region').precip.mean()
regional

In [ ]:
regional.plot(kind='bar')

This is a lot easier to read if it's sorted:

In [ ]:
regional.sort(inplace=False).plot(kind='bar')


(Check this map if you're curous where these different regions are: http://www.oocities.org/watergeographer/hydromap.gif.)

If you have a DataFrame of data with the same kind of data in each column you can use the .plot method to create plots with different lines (or other styles) for each column. But first we need to make a DataFrame with data laid out in that way. Let's compare average rainfall in each region over time, which requires grouping on year and region then calculating the mean:

In [ ]:
regional_yearly = monthly.groupby(['year', 'region']).precip.mean()
regional_yearly.head(15)

regional_yearly is a Series with a two-level index (a MultiIndex), one for year and one for region. What we're after is a DataFrame with regions as columns and precip values for each region/year in the table. The Series .unstack() method can be used to pivot index labels into column labels:

In [ ]:
regional_yearly = regional_yearly.unstack(level='region')
regional_yearly.head()

Now we can use .plot to get a different line on the plot for every region (column):

In [ ]:
regional_yearly.plot(kind='line', figsize=(10, 8), colormap='Set3', linewidth=4)